## <a name="0">Finetuning Large Language Model on Amazon SageMaker - DIY</a>

**This notebook uses Amazon Bedroc FAQ data to fine-tune an LLM.**

**Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.**


## <a name="step1">Step 1: Check GPU Memory</a>

To check the GPU memory, run the below command.

In [ ]:
!nvidia-smi

**If you see your CUDA memory is occupied by more than half (like the image below), then you need to shutdown other running notebooks.**

<p style="padding: 10px; border: 1px solid black;">
<img src="images/memory.png" alt="drawing" width="800"/> <br/>

## <a name="step2">Step 2: Import libraries</a>

Run the below code block sequentially to import the necessary libraries, including the Hugging Face Transformers library and the PyTorch library, which is a dependency for Transformers. If you haven't install the packages, uncomment the below line and install them.


In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
# %%capture

import warnings
warnings.filterwarnings('ignore')  # Disables all warnings

import os
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Tuple, Union
from datasets import Dataset, load_dataset, disable_caching
disable_caching() ## disable huggingface cache

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import TextDataset

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer
import accelerate
import bitsandbytes

from IPython.display import Markdown

## <a name="step3">Step 3: Prepare the training dataset</a>

Load and view the dataset. You will use [Amazon Bedrock FAQs](https://aws.amazon.com/bedrock/faqs/) as the main dataset. The dataset has two columns `instruction` and `response`.

In [ ]:
bedrock_faqs_dataset = load_dataset("csv",
                                      data_files='data/amazon_bedrock_faqs.csv')['train']
bedrock_faqs_dataset

In [ ]:
bedrock_faqs_dataset[0]

## <a name="step3">Step 3.1: Prepare the Prompt</a>
To finetune our LLM, you need to decorate the instruction dataset with a PROMPT like below.

In [ ]:
from utils.helpers import INTRO_BLURB, INSTRUCTION_KEY, RESPONSE_KEY, END_KEY, RESPONSE_KEY_NL, DEFAULT_SEED, PROMPT
'''
PROMPT = """{intro}
            {instruction_key}
            {instruction}
            {response_key}
            {response}
            {end_key}"""
'''
Markdown(PROMPT)

Now feed the PROMPT to the dataset via the below function named `_add_text`. It takes a record as input. The function first checks if both the instruction and response fields are not empty. If either of them is empty, it raises a ValueError with a corresponding error message. If both fields have values, the function creates a new "text" field in the record by formatting them using given PROMPT.

In [ ]:
def _add_text(rec):
    instruction = rec["instruction"]
    response = rec["response"]

    if not instruction:
        raise ValueError(f"Expected an instruction in: {rec}")

    if not response:
        raise ValueError(f"Expected a response in: {rec}")

    rec["text"] = PROMPT.format(
        instruction=instruction, response=response)

    return rec

In [ ]:
bedrock_faqs_dataset = bedrock_faqs_dataset.map(_add_text)
bedrock_faqs_dataset[0]

Use `Markdown` to neatly display the text with PROMPT.

In [ ]:
Markdown(bedrock_faqs_dataset[0]['text'])

### <a name="#step4">Step 4: Load a pretrained LLM</a>


To load a pretrained model, initialize a tokenizer and a base model using the `databricks/dolly-v2-3b` model from the Hugging Face Transformers library. The tokenizer converts raw text into tokens, and the base model generates text based on a given prompt. By following the instructions outlined above, you can correctly instantiate these components and leverage their functionality in your code.


The `AutoTokenizer.from_pretrained()` function is used to instantiate the tokenizer.
- `padding_side="left"` specifies the side of the sequences where padding tokens will be added. In this case, padding tokens will be added to the left side of each sequence.
- The `eos_token` is a special token representing the end of a sequence. By assigning it to the `pad_token`, any padding tokens added during tokenization will also be considered as end-of-sequence tokens. This can be useful when generating text using the model, as it will know when to stop generating text after encountering padding tokens.
- The `tokenizer.add_special_tokens...` adds three additional special tokens to the tokenizer's vocabulary. These tokens likely serve specific purposes in the application using the tokenizer. For example, they could be used to mark the end of an input, an instruction, or a response in a dialogue system.

After execution, the `tokenizer` object will be initialized and ready to use for tokenizing text.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("verseAI/databricks-dolly-v2-3b",
                                          padding_side="left")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens":
                              [END_KEY, INSTRUCTION_KEY, RESPONSE_KEY_NL]})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "verseAI/databricks-dolly-v2-3b",
    # use_cache=False,
    torch_dtype=torch.float16,
    device_map="auto", #"balanced",
    load_in_8bit=True,
)

### <a name="#step4.1">Step 4.1: Prepare model for training</a>
Some pre-processing needs to be done before training such an int8 model using peft, therefore import an utiliy function `prepare_model_for_int8_training` that will:

- Casts all the non `int8` modules to full precision (fp32) for stability
- Add a forward_hook to the input embedding layer to enable gradient computation of the input hidden states
- Enable gradient checkpointing for more memory-efficient training

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Use the `preprocess_batch` function to preprocess the "text" field of the batch, applying tokenization, truncation, and other relevant operations based on the specified maximum length. It takes a batch of data, a tokenizer, and a maximum length as input.

Refer to `mlu_utils/helpers.py` file for more details.

In [ ]:
from functools import partial
from utils.helpers import mlu_preprocess_batch

MAX_LENGTH = 256
_preprocessing_function = partial(mlu_preprocess_batch, max_length=MAX_LENGTH, tokenizer=tokenizer)

Next, apply the preprocessing function to each batch in the dataset, modifying the "text" field accordingly. The map operation is performed in a batched manner and the "instruction", "response", and "text" columns are removed from the dataset. Finally, the `processed_dataset` is created by filtering the `bedrock_faqs_dataset` based on the length of the "input_ids" field, ensuring it is less than the specified `MAX_LENGTH`.

In [ ]:
encoded_bedrock_faqs_dataset = bedrock_faqs_dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "response", "text"],
)

processed_dataset = encoded_bedrock_faqs_dataset.filter(lambda rec: len(rec["input_ids"]) < MAX_LENGTH)

Split dataset into `train` and `test` for evaluation.

In [ ]:
split_dataset = processed_dataset.train_test_split(test_size=14, seed=0)
split_dataset

### <a name="#step5">Step 5: Define the trainer and finetuned the LLM</a>

To finetune a model efficiently, You will use [LoRA: Low-Rank Adaptation](https://arxiv.org/abs/2106.09685). LoRA, which freezes the pre-trained model weights and injects trainable rank decomposition matrices into each layer of the Transformer architecture, greatly reducing the number of trainable parameters for downstream tasks. Compared to GPT-3 175B fine-tuned with Adam, LoRA can reduce the number of trainable parameters by 10,000 times and the GPU memory requirement by 3 times.


#### <a name="#step5.1">Step 5.1: Define the `LoraConfig` and load LoRA model</a>

You'll us build LoRA class `LoraConfig` from [huggingface 🤗 PEFT: State-of-the-art Parameter-Efficient Fine-Tuning](https://github.com/huggingface/peft). Within `LoraConfig`, let's specify the following parameters:

- `r`, the dimension of the low-rank matrices
- `lora_alpha`, the scaling factor for the low-rank matrices
- `lora_dropout`, the dropout probability of the LoRA layers

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# First prepare the model for int8 training
model = prepare_model_for_int8_training(model)

# Then freeze all parameters
for param in model.parameters():
    if param.dtype == torch.float32 or param.dtype == torch.float16:
        param.requires_grad = False

# Unfreeze only the top N layers
num_layers_to_unfreeze = 2  # Adjust this number as needed
for i, layer in enumerate(model.gpt_neox.layers[-num_layers_to_unfreeze:]):
    for param in layer.parameters():
        if param.dtype == torch.float32 or param.dtype == torch.float16:
            param.requires_grad = True
            
MICRO_BATCH_SIZE = 8
BATCH_SIZE = 64
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LORA_R = 256 # 512
LORA_ALPHA = 512 # 1024
LORA_DROPOUT = 0.05

# Define LoRA Config
lora_config = LoraConfig(
                 r=LORA_R,
                 lora_alpha=LORA_ALPHA,
                 lora_dropout=LORA_DROPOUT,
                 bias="none",
                 task_type="CAUSAL_LM"
)

Use the `get_peft_model` function to initialize the model with the LoRA framework, configuring it based on the provided `lora_config` settings. This allows the model to incorporate the benefits and capabilities of the LoRA optimization approach.

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

As you can see above, LoRA only trainable parameters is only ~3% of the full weights. Much efficient!

#### <a name="#step5.2">Step 5.2: Define the data collator</a>

A DataCollator is a huggingface🤗 transformers function that takes a list of samples from a Dataset and collate them into a batch, as a dictionary of PyTorch tensors.

Use `DataCollatorForCompletionOnlyLM`, which extends the functionality of the base `DataCollatorForLanguageModeling` class from transformers library. This custom collator is designed to handle examples where a prompt is followed by a response in the input text and modifies the labels accordingly.

Refer to `utils/helpers.py` for the implementation.

In [ ]:
from utils.helpers import MLUDataCollatorForCompletionOnlyLM

data_collator = MLUDataCollatorForCompletionOnlyLM(
        tokenizer=tokenizer, mlm=False, return_tensors="pt", pad_to_multiple_of=8
)

#### <a name="#step5.3">Step 5.3: Define the trainer</a>

To finetune the LLM, you need to define a trainer. First, define a training arguments.

In [ ]:
EPOCHS = 10
LEARNING_RATE = 1e-4
MODEL_SAVE_FOLDER_NAME = "diy-dolly-3b-lora"

training_args = TrainingArguments(
                    output_dir=MODEL_SAVE_FOLDER_NAME,
                    fp16=True,
                    per_device_train_batch_size=1,
                    per_device_eval_batch_size=1,
                    learning_rate=LEARNING_RATE,
                    num_train_epochs=EPOCHS,
                    logging_strategy="steps",
                    logging_steps=100,
                    evaluation_strategy="steps",
                    eval_steps=100,
                    save_strategy="steps",
                    save_steps=20000,
                    save_total_limit=10,
                    optim="adamw_torch"
)

Now is where the magic happen! Initialize the trainer with the defined model, tokenizer, training arguments, data collator and the train/eval datasets.

The training may take ~10 minutes. Once the training is done, you will save the finetuned model and tokenizer.

In [ ]:
trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=split_dataset['train'],
        eval_dataset=split_dataset["test"],
        data_collator=data_collator,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

#### <a name="#step5.4">Step 5.4: Save the finetuned model</a>


After the training finished, you can save the model to a directory using the [`transformers.PreTrainedModel.save_pretrained`] function.
This function only saves the incremental 🤗 PEFT weights (adapter_model.bin) that were trained, meaning it is super efficient to store, transfer, and load.

In [ ]:
trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)

If you want to save the full model you just finetuned, you can simply use the [`transformers.trainer.save_model`] function. Meanwhile, save the training arguments together with the trained model.

In [ ]:
trainer.save_model()

In [ ]:
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

Let's save the tokenizer along with the trained model

In [ ]:
tokenizer.save_pretrained(MODEL_SAVE_FOLDER_NAME)

### <a name="#step6">Step 6: Deploy the fine tuned model</a>

### <a name="step6">Overview of deployment parameters</a>

To deploy using the SageMaker Python SDK with the DJL, you will need to instantiate `Model` class with the following parameters:
```{python}
model = Model(
    image_uri,
    model_data=...,
    predictor_cls=...,
    role=aws_role
)
```
- `image_uri`: The Docker image URI representing the deep learning framework and version to be used.
- `model_data`: The location of the finetuned LLM model artifact in an S3 bucket. It specifies the path to the TAR GZ file containing the model's parameters, architecture, and any necessary artifacts.
- `predictor_cls`: This is just a "json in json out" predictor, nothing DJL related, check more details at [sagemaker.djl_inference.DJLPredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/djl/sagemaker.djl_inference.html#djlpredictor).
- `role`: The AWS Identity and Access Management (IAM) role ARN that provides necessary permissions to access resources like the S3 bucket containing the model data.

### <a name="step6.1">Step 6.1: Instantiate SageMaker parameters</a>

Initialize a SageMaker session and retrieve information related to the AWS environment such as SageMaker role and AWS region. You also specify the image URI for a specific version of the "djl-deepspeed" framework using the SageMaker session's region. The image URI is a unique identifier for a specific Docker container image that can be used in various AWS services, such as Amazon SageMaker or Elastic Container Registry (ECR).

In [ ]:
# installing sagemaker library
!pip3 install sagemaker==2.237.1

In [ ]:
import boto3
import json
import sagemaker.djl_inference
from sagemaker.session import Session
from sagemaker import image_uris
from sagemaker import Model

sagemaker_session = Session()
print("sagemaker_session: ", sagemaker_session)

aws_role = sagemaker_session.get_caller_identity_arn()
print("aws_role: ", aws_role)

aws_region = boto3.Session().region_name
print("aws_region: ", aws_region)

image_uri = image_uris.retrieve(framework="djl-deepspeed",
                                version="0.22.1",
                                region=sagemaker_session._region_name)
print("image_uri: ", image_uri)


### <a name="step6.2">Step 6.2: Create the model artifact</a> ###

To upload the model artifact in the S3 bucket, we need to create TAR GZ file containing the model's parameters. First, you create a directory named `lora_model` and a subdirectory named `dolly-3b-lora`. The "-p" option ensures that the command creates any intermediate directories if they don't exist. Then, you copy the lora checkpoints `adapter_model.bin` and `adapter_config.json` to `dolly-3b-lora`. The base dolly model will be downloaded at runtime from huggingface hub.

In [ ]:
%%bash
rm -rf lora_model
mkdir -p lora_model
mkdir -p lora_model/dolly-3b-lora
cp diy-dolly-3b-lora/adapter_config.json lora_model/dolly-3b-lora/
cp diy-dolly-3b-lora/adapter_model.bin lora_model/dolly-3b-lora/

Next, set the [DJL Serving configuration options](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html) in `serving.properties`. The jupyter `%%writefile` magic command enables us to write the following content to a file named "lora_model/serving.properties".
- `engine=Python`: This line specifies the engine used for serving.
- `option.entryPoint=model.py`: This line specifies the entry point for the serving process, which is set to "model.py".
- `option.adapter_checkpoint=dolly-3b-lora`: This line sets the checkpoint for the adapter to "dolly-3b-lora". A checkpoint typically represents the saved state of a model or its parameters.
- `option.adapter_name=dolly-lora`: This line sets the name of the adapter to "dolly-lora", a component that helps interface between the model and the serving infrastructure.

In [ ]:
%%writefile lora_model/serving.properties
engine=Python
option.entryPoint=model.py
option.adapter_checkpoint=dolly-3b-lora
option.adapter_name=dolly-lora

Another file you need in the the model artifact is the environment requirement file. Create a file named `lora_model/requirements.txt` and write a list of Python package requirements, typically used with package managers like `pip`.

In [ ]:
%%writefile lora_model/requirements.txt
accelerate>=0.16.0,<1
bitsandbytes==0.39.0
click>=8.0.4,<9
datasets>=2.10.0,<3
deepspeed>=0.8.3,<0.9
faiss-cpu==1.7.4
ipykernel==6.22.0
scipy==1.11.1
torch>=2.0.0
transformers==4.28.1
peft==0.3.0
pytest==7.3.2

### <a name="step6.3">Step 6.3: Create the inference script</a>

Similar to the finetuning notebook, a custom pipeline `InstructionTextGenerationPipeline` is defined. The code is provided in `utils/deployment_model.py`.

You save these inference functions to `lora_model/model.py`.

In [ ]:
%%bash
cp utils/deployment_model.py lora_model/model.py

### <a name="step6.4">Step 6.4: Upload the model artifact to S3</a>

Create a compressed tarball archive of the "lora_model" directory and saves it as "lora_model.tar.gz".

In [ ]:
%%bash
tar -cvzf diy_lora_model.tar.gz lora_model/

Upload the "lora_model.tar.gz" file to the specified S3 bucket.

In [ ]:
import boto3
import json
import sagemaker.djl_inference
from sagemaker.session import Session
from sagemaker import image_uris
from sagemaker import Model

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

s3 = boto3.resource('s3')

# Get the name of the bucket with prefix lab-code
for bucket in s3.buckets.all():
    if bucket.name.startswith('artifact'):
        mybucket = bucket.name
        print(mybucket)

response = s3_client.upload_file("diy_lora_model.tar.gz", mybucket, "diy_lora_model.tar.gz")

### <a name="step6.5">Step 6.5: Deploy the Model</a> ###

Now it's the time to deploy the finetuned LLM using SageMaker Python SDK. The SageMaker Python SDK `Model` class is instantiated with the following parameters:

- `image_uri`: The Docker image URI representing the deep learning framework and version to be used.
- `model_data`: The location of the finetuned LLM model artifact in an S3 bucket. It specifies the path to the TAR GZ file containing the model's parameters, architecture, and any necessary artifacts.
- `predictor_cls`: This is just a "json in json out" predictor, nothing DJL related, check more details at [sagemaker.djl_inference.DJLPredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/djl/sagemaker.djl_inference.html#djlpredictor).
- `role`: The AWS Identity and Access Management (IAM) role ARN that provides necessary permissions to access resources like the S3 bucket containing the model data.
- `name`: The model name. If None, a default model name will be selected on each deploy.

In [ ]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_data="s3://{}/diy_lora_model.tar.gz".format(mybucket)
model_name=f"diy-model-{timestamp_prefix}"

model = Model(image_uri=image_uri,
              name = model_name,
              model_data=model_data,
              predictor_cls=sagemaker.djl_inference.DJLPredictor,
              role=aws_role)

Note: **The deployment should finish within 10 minutes. If it took longer than that, your endpoint may be failed.**

In [ ]:
%%time

#Define the unique name for the endpoint
endpoint_name = f"diy-endpoint-{timestamp_prefix}"

predictor = model.deploy(1, "ml.g5.2xlarge", endpoint_name=endpoint_name )

### <a name="step7">Step 7: Test the deployed Inference</a>

Test the inference endpoint with [predictor.predict](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor.predict)!

In [ ]:
outputs = predictor.predict({"inputs": "Is the content processed by Amazon Bedrock moved outside the AWS Region where I am using Amazon Bedrock?"})

In [ ]:
from IPython.display import Markdown
Markdown(outputs)